In [1]:
import os
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout, LayerNormalization

In [2]:
def normalize_arabic(text):
    text = re.sub(r'[ًٌٍَُِّْـ]', '', text)
    text = re.sub(r'[إأآا]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ة', 'ه', text)
    return text

def split_into_sentences(text):
    return [sentence.strip() for sentence in re.split(r'[.!؟\n]', text) if sentence.strip()]

In [3]:
arabic_keyboard = {
    'ا': ['أ', 'إ', 'ء', 'ى'],
    'ب': ['ن', 'ت'],
    'ت': ['ب', 'ن'],
    'ث': ['ت', 'س'],
    'ج': ['ح', 'خ'],
    'ح': ['ج', 'خ'],
    'خ': ['ح', 'ج'],
    'د': ['ذ'],
    'ذ': ['د', 'ر'],
    'ر': ['ذ', 'ز'],
    'ز': ['ر', 'س'],
    'س': ['ش', 'ص'],
    'ش': ['س', 'ص'],
    'ص': ['س', 'ش', 'ض'],
    'ض': ['ص', 'ط'],
    'ط': ['ض', 'ظ'],
    'ع': ['غ'],
    'غ': ['ع'],
    'ف': ['ق'],
    'ق': ['ف'],
    'ك': ['ل'],
    'ل': ['ك'],
    'م': ['ن'],
    'ن': ['م', 'ب'],
    'ه': ['ة'],
    'ة': ['ه'],
    'ي': ['ى', 'ب'],
    'ى': ['ي']
}

def keyboard_substitute(char):
    if char in arabic_keyboard:
        return random.choice(arabic_keyboard[char])
    return char

def insert_typo(word):
    if len(word) < 2:
        return word
    # typo_type = random.choice(['delete', 'insert', 'substitute'])
    typo_type = 'substitute'
    i = random.randint(0, len(word) - 1)
    arabic_chars = 'ابتثجحخدذرزسشصضطظعغفقكلمنهوي'

    if typo_type == 'delete':
        return word[:i] + word[i+1:]
    elif typo_type == 'insert':
        return word[:i] + random.choice(arabic_chars) + word[i:]
    elif typo_type == 'substitute':
        return word[:i] + keyboard_substitute(word[i]) + word[i+1:]
    return word

def corrupt_sentence(sentence, error_probability=0.1):
    words = sentence.split()
    corrupted = [insert_typo(w) if random.random() < error_probability else w for w in words]
    return ' '.join(corrupted)

In [4]:
df = pd.read_csv('unbalanced_reviews.tsv', sep='\t', header=None)

texts = df[4].astype(str).tolist()

all_sentences = []
for text in texts:
    norm_text = normalize_arabic(text)
    all_sentences.extend(split_into_sentences(norm_text))

clean_sentences = [s for s in all_sentences if 5 < len(s) < 100]
clean_sentences = list(set(clean_sentences))  # Remove duplicates
noisy_sentences = [corrupt_sentence(s) for s in clean_sentences]

print(f"Clean samples: {len(clean_sentences)}")

Clean samples: 2182769


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510599 entries, 0 to 510598
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   0       510599 non-null  int64 
 1   1       510599 non-null  int64 
 2   2       510599 non-null  int64 
 3   3       510599 non-null  int64 
 4   4       510599 non-null  object
dtypes: int64(4), object(1)
memory usage: 19.5+ MB


In [6]:
df.head()

,0,1,2,3,4
0,4,1682581870,57098525,13637412,صراع الجذور والانتماء، عقلة ساق الخيزان توائم ...
1,5,1682385404,56693085,13637412,كتاب رائع. اعتقد ان الروايه كلها تلخصت بجمله و...
2,4,1682039752,30836455,13637412,رواية تلامس الروح بعمقها، فخورة اني اخيرا لقيت...
3,5,1681553886,6680940,13637412,رواية محكمة بكل اختصار. وكان الجزء المفضل بالن...
4,3,1681248984,19011044,13637412,هذا الكتاب يحزن مرا، ظلم واضطهاد عيسى بلا ذنب ...


In [7]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(clean_sentences + noisy_sentences)

vocab_size = len(tokenizer.word_index) + 1
max_len = max(len(s) for s in clean_sentences)

def encode(sentences):
    sequences = tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=max_len, padding='post')

X = encode(noisy_sentences)
Y = encode(clean_sentences)
Y = Y[..., None] 

In [8]:
embedding_dim = 300
lstm_units = 256
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len, input_shape=(max_len,)),
    Bidirectional(LSTM(lstm_units, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(128, return_sequences=True)),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 99, 300)           50400     
_________________________________________________________________
bidirectional (Bidirectional (None, 99, 512)           1140736   
_________________________________________________________________
dropout (Dropout)            (None, 99, 512)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 99, 256)           656384    
_________________________________________________________________
time_distributed (TimeDistri (None, 99, 168)           43176     
Total params: 1,890,696
Trainable params: 1,890,696
Non-trainable params: 0
_________________________________________________________________


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.models import load_model

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6
)

early_stop = EarlyStopping(
    patience=5,
    restore_best_weights=True,
    monitor='val_accuracy',
    mode='max'
)

checkpoint = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

model.fit(
    X_train, Y_train,
    epochs=1,
    batch_size=64,
    validation_data=(X_test, Y_test),
    callbacks=[checkpoint, early_stop, reduce_lr]
)

27285/27285 [==============================] - 48684s 2s/step - loss: 0.0226 - accuracy: 0.9948 - val_loss: 0.0098 - val_accuracy: 0.9977

Epoch 00001: val_accuracy improved from -inf to 0.99774, saving model to best_model.keras


In [10]:
model = load_model('best_model.keras')

def decode_sequence(pred):
    pred_ids = np.argmax(pred, axis=-1)
    index_word = {v: k for k, v in tokenizer.word_index.items()}
    return ''.join([index_word.get(i, '') for i in pred_ids])

def autocorrect(input_text):
    input_text = normalize_arabic(input_text)
    seq = encode([input_text])
    pred = model.predict(seq)
    return decode_sequence(pred[0])

In [11]:
sentences_with_typos = [
    "أنا أجب المدرسة",     # أحب → أجب
    "هو يقأ كتابًا",       # يقرأ → يقأ
    "الحو جميل اليوم",     # الجو → الحو
    "ذهبت إلى السىق",      # السوق → السىق
    "هي تطبخ الطعان",      # الطعام → الطعان
    "القط يحلس هناك",      # يجلس → يحلس
    "الولد يكتب الواخب",   # الواجب → الواخب
    "أين حخيبتي؟",         # حقيبتي → حخيبتي
    "أنا متعب قببلاً",     # قليلاً → قببلاً
    "نحن نلعب في الجديقة", # الحديقة → الجديقة
    "السماء زرقاء صافبة",  # صافية → صافبة
    "هل أكلت القبور؟",     # الفطور → القبور
    "أريد شرب الهاء",      # الماء → الهاء
    "السيارة سريعة جذاً",  # جداً → جذاً
    "الوقت متأحر الآن",    # متأخر → متأحر
    "أحب كرة القدن",       # القدم → القدن
    "هو يعمل بحد",         # بجد → بحد
    "الطفل نائك",          # نائم → نائك
    "أين المغاتيح؟",       # المفاتيح → المغاتيح
    "الشارع مزذحم"         # مزدحم → مزذحم
]
for test_input in sentences_with_typos:
    print("Noisy input:    ", test_input)
    print("Autocorrected:  ", autocorrect(test_input))

Noisy input:     أنا أجب المدرسة
Autocorrected:   انا احب المدرسه
Noisy input:     هو يقأ كتابًا
Autocorrected:   هو يقا كتابا
Noisy input:     الحو جميل اليوم
Autocorrected:   الجو جميل اليوم
Noisy input:     ذهبت إلى السىق
Autocorrected:   ذهبت الي السيق
Noisy input:     هي تطبخ الطعان
Autocorrected:   هي تطبخ الطعام
Noisy input:     القط يحلس هناك
Autocorrected:   القط يحلس هناك
Noisy input:     الولد يكتب الواخب
Autocorrected:   الولد يكتب الواحب
Noisy input:     أين حخيبتي؟
Autocorrected:   اين حخيبتي
Noisy input:     أنا متعب قببلاً
Autocorrected:   انا متعب قببلا
Noisy input:     نحن نلعب في الجديقة
Autocorrected:   نحن نلعب في الحديقه
Noisy input:     السماء زرقاء صافبة
Autocorrected:   السماء زرقاء صافيه
Noisy input:     هل أكلت القبور؟
Autocorrected:   هل اكلت القبور
Noisy input:     أريد شرب الهاء
Autocorrected:   اريد شرب الهاء
Noisy input:     السيارة سريعة جذاً
Autocorrected:   السياره سريعه جدا
Noisy input:     الوقت متأحر الآن
Autocorrected:   الوقت متاحر الان
Noisy inp

In [12]:
from difflib import SequenceMatcher

def sentence_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

scores = [sentence_similarity(clean, autocorrect(noisy)) for clean, noisy in zip(clean_sentences[:100], noisy_sentences[:100])]
print(f"Avg sentence similarity: {np.mean(scores):.4f}")

Avg sentence similarity: 0.9947


In [13]:
def evaluate_sentence_level_accuracy(clean_list, noisy_list):
    correct_count = 0
    total = len(clean_list)
    for clean, noisy in zip(clean_list, noisy_list):
        prediction = autocorrect(noisy)
        if prediction == clean:
            correct_count +=1
        print(clean)
        print(noisy)
    print(f"Sentence-level accuracy: {correct_count / total:.4f}")

evaluate_sentence_level_accuracy(clean_sentences[500:700], noisy_sentences[500:700])

وها نحن فقدنا انسانيتنا وهويتنا فصرنا بلا قضيه
وها نحن فقدنا انسانيتنا وهويتنا فصرنا بلء قضيه
و لكنه احيانا ينسي هذا المنهج حين يسترسل في الكتابه و نسج الفكره
و لكنه احيانا ينسي هذا المنهج حين ىسترسل في الكتابه و مسج الفكره
كتاب رايع و مفيد و جامع لليسر و البساطه في الطرح و وضوح الاسلوب
كتاب زايع و مفيد و جامع لليسر و البساطه في الطرح و وضوح الاسلوب
كتاب جميل انصح بقراءته يلخص الكثير من الافكار
كتاب جميل انصح بقراءته يلخص الكثير من الىفكار
بمحاذاه الشاطي يصنع فلاسفه لكن الابحار يصنع قصه حياه
بمحاذاه الشاطي يصنع فلاسفه لكن الابحار يصنع قصه حياه
ليس من اراد الحق فاخطاه كمن اراد الباطل فاصابه
ليس من اذاد الحق فاخطاه كمن اراد الباطل فاصابه
استمر يا د
استمر يا د
منذ ذلك اليوم وانا انتظر عوده تلك الطايره التي قد تضرب بيروت في ايه لحظه
منذ ذلك اليوم وانا انتظر عوده تلك الطايره التي قد تضرب بيروت في ايه لحظه
الحجاره اللي ف ايدين ثوار فلسطين الصغار
الحجاره اللي ف ايدين ثوار فلسطين الصغار
الجمال الحقيقي هو اشعه تنبعث من قدس
الجمال الحقيقي هو اسعه تنبعث مم قدس
ثمره تجربه الدكتور طارق سويدان في ال

In [14]:
def evaluate_autocorrect_model(clean_sentences, noisy_sentences, verbose=True):
    assert len(clean_sentences) == len(noisy_sentences), "Mismatched input lengths."
    
    sentence_correct = 0
    word_accuracies = []
    
    for clean, noisy in zip(clean_sentences, noisy_sentences):
        predicted = autocorrect(noisy)
        
        if predicted == clean:
            sentence_correct += 1
        
        clean_words = clean.split()
        predicted_words = predicted.split()
        correct_words = sum(1 for cw, pw in zip(clean_words, predicted_words) if cw == pw)
        word_accuracy = correct_words / max(len(clean_words), 1)
        word_accuracies.append(word_accuracy)

        print("🔸 Noisy:     ", noisy)
        print("🔁 Predicted:", predicted)
        print("✅ Target:   ", clean)
        if predicted == clean and noisy != clean: 
            print("Corrected a mistake!")
        if predicted == clean and noisy == clean: 
            print("Correct, but no mistake was found")
        print()

    total = len(clean_sentences)
    print(f"\n📏 Sentence-level accuracy: {sentence_correct}/{total} = {sentence_correct/total:.2%}")
    print(f"📊 Avg. word-level accuracy: {np.mean(word_accuracies):.2%}")

In [19]:
sentence1 = "أنا أجب المدرصة"
sentence2 = "أنا أشعر بالحوع"
sentence3 = "الحديقة حميلة جدًا"
sentence4 = "أدرس اللغة العزبية"
sentence5 = "أحب أتغلم البرمجة"
sentence6 = "السماء ززقاء صافية"
sentence7 = "مغالجة اللغات الطبيعية"
print(autocorrect(sentence1))

انا احب المدرسه


In [18]:
import tkinter as tk
def correct_sentence():
    input_text = entry.get()
    corrected = autocorrect(input_text)
    result_label.config(text=" التصحيح: " + corrected)

root = tk.Tk()
root.title("Arabic Sentence Autocorrector")
root.geometry("500x200")

label = tk.Label(root, text="أدخل الجملة العربية:", font=("Arial", 14))
label.pack(pady=10)

entry = tk.Entry(root, font=("Arial", 14), justify='right')
entry.pack(fill='x', padx=20)

button = tk.Button(root, text="تصحيح", font=("Arial", 12), command=correct_sentence)
button.pack(pady=10)

result_label = tk.Label(root, text="", font=("Arial", 14), fg="green", wraplength=480, justify='right')
result_label.pack(pady=10)

root.mainloop()